# Nenana Ice Classic Data Processing
This notebook was used to process the data gathered for this project.

In [1]:
# imports

# data
import numpy as np
import pandas as pd

# date processing
import datetime

# filter warnings
import warnings
warnings.filterwarnings("ignore")

# garbage collection
import gc

## Helper Functions

In [2]:
def slope(y):
    x = range(len(y))
    # y = mx + b
    m, b = np.polyfit(x, y, 1)
    return m

def accel(y):
    t = range(len(y))
    # y = 1/2 g t^2 + v t + y0
    a, v, y0 = np.polyfit(t, y, 2)
    return 5*a

In [3]:
def gap_fill_mean(data, missing_val_field):
    '''
    This function fills in missing values in a DataFrame column with the mean of the values before
    and after a missing value.
    '''
    missing_idx = []
    start_idx, mean_val, end_idx = 0, 0, 0
    found_start, found_end = False, False
    for idx in data.index:
        if np.isnan(data[missing_val_field].loc[idx]) == True:
            missing_idx.append(idx)
            # if missing value at index, check if index is first or last entry in data
            # if not first or last, check previous and next records for values
            if idx != data.index[0] and idx != data.index[-1]:
                if np.isnan(data[missing_val_field].loc[idx - 1]) == False:
                    start_idx = idx - 1
                    found_start = True
                if np.isnan(data[missing_val_field].loc[idx + 1]) == False:
                    end_idx = idx + 1
                    found_end = True
            else:
                # if first entry is missing
                if idx == data.index[0]:
                    # check 2nd entry; if 2nd exists, make that index the end
                    if np.isnan(data[missing_val_field].loc[idx + 1]) == False:
                        end_idx = idx + 1
                        found_end = True
                    else:
                        # if 2nd entry also missing, end index will be found but not start
                        pass # handle later
                else:
                    # if last entry is missing, check next to last; if exists, make that start & end
                    if np.isnan(data[missing_val_field].loc[idx - 1]) == False:
                        start_idx = idx - 1
                        found_start = True
                    else:
                        # if next to last also missing, start index will be found but not end
                        pass # handle later
        # normal process, start and end indices are found
        if found_start and found_end:
            mean_val = np.mean([data[missing_val_field].loc[start_idx], data[missing_val_field].loc[end_idx]])
            for upd_idx in range(start_idx + 1, end_idx):
                data[missing_val_field].loc[upd_idx] = mean_val
            found_start, found_end = False, False
            start_idx, mean_val, end_idx = 0, 0, 0
        # values missing at start of data, end index was found but not start
        elif found_end and not found_start:
            # update values from start of data until end index
            for upd_idx in range(data.index[0], end_idx):
                data[missing_val_field].loc[upd_idx] = data[missing_val_field].loc[end_idx]
            found_end = False
            end_idx = 0
        # value(s) missing at end of data
        elif idx == data.index[-1] and found_start and not found_end:
            # update values from start index until end of data
            for upd_idx in range(start_idx + 1, data.index[-1]):
                data[missing_val_field].loc[upd_idx] = data[missing_val_field].loc[start_idx]
            # set last entry to most recent known value
            data[missing_val_field].loc[data.index[-1]] = data[missing_val_field].loc[start_idx]
            found_start = False
            start_idx = 0

## Reading data from files

In [ ]:
ice_df = pd.read_csv('../data/raw_ice_thickness_1989-2019.csv')
ice_df.head()

In [ ]:
ice_df.drop(columns = 'Date', inplace = True)

In [ ]:
ice_df.rename(columns = {'Date.1' : 'Date', 'Thickness' : 'iceThickness'}, inplace = True)

In [ ]:
ice_df.shape

In [ ]:
ice_df.info()

In [ ]:
weather_df = pd.read_csv('../data/raw_weather_1989-2020.csv')
weather_df.head()

In [ ]:
weather_df.rename(columns = {'time' : 'readingTime',
                             'summary' : 'weatherSummary',
                             'pressure' : 'atmoPressure'}, inplace = True)

In [ ]:
weather_df.shape

In [ ]:
weather_df.info()

In [ ]:
winners_df = pd.read_csv('../data/ice_classic_winning_times.csv')
winners_df.head()

In [ ]:
winners_df.rename(columns = {'Time' : 'winningTime'}, inplace = True)

In [ ]:
winners_df.shape

In [ ]:
winners_df.info()

In [ ]:
river_df = pd.read_csv('../data/river_flow_data.csv')
river_df.head()

In [ ]:
river_df.drop(columns = ['dateTime', 'qualifiers'], inplace = True)

In [ ]:
river_df.rename(columns = {'dateTime.1' : 'Date', 'value' : 'flowVolume'}, inplace = True)

In [ ]:
river_df.shape

In [ ]:
river_df.info()

## Merge the DataFrames into one

In [ ]:
merged_df = weather_df.merge(winners_df, how = 'left', on = 'Date')
merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
merged_df.info()

In [ ]:
merged_df = merged_df.merge(ice_df, how = 'left', on = 'Date')
merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
merged_df.info()

In [ ]:
df = merged_df.merge(river_df, how = 'left', on = 'Date')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# delete unneeded variables
del merged_df, ice_df, winners_df, weather_df, river_df
gc.collect()

#### Convert Date And Time Columns To Datetime Object

In [ ]:
# convert Date from string to datetime object
df['Date'] = pd.to_datetime(df['Date'], yearfirst = True, infer_datetime_format = True)
df['winningTime'] = pd.to_datetime(df['winningTime'], infer_datetime_format = True, errors = 'ignore')

### Dealing With Missing Values

---
#### (event)Time, apparentTemperature(category), weatherSummary, precipProbability, precipIntensityMax, visibility, temperatureHigh, temperatureLow, dewPoint, icon, ozone, windGust
I didn't think any of these would help with this prediction task.

My reasoning:
* Most "(event)Time" features dropped were because they only captured a single, specific event (the time the daytime high temperature was recorded, for example)
  * I may go back and get hourly data; something like length of time that the temperature was high or low for a day might have an impact. For now, I passed this by due to time constraints.
* "apparentTemperature(category)" features were dropped because they're a representation of what conditions feel like, not what they actually are.
* "weatherSummary"  was dropped because it's a text summary of information captured by other features.
* "precipProbability" was dropped because this is past data and the presence/amount of precipitation is known.
* "precipIntensityMax" was dropped because the precipitation rate is already captured by "precipIntensity" and I don't think knowing the maximum precipitation rate adds anything.
* "visibility" was dropped because atmospheric conditions are already captured by other features (e.g., "uvIndex," "precipIntensity")
* "temperatureHigh" and "temperatureLow" were dropped because the first is the daytime high and the second is the nighttime low; the 24 hour maximum and minimum are captured by "temperatureMax" and "temperatureMin" respectively
* "dewPoint" was dropped because it's correlated with temperature and humidity
* "icon" was dropped because it was missing values in just under half of the columns and it didn't appear to capture any unique information that wasn't already captured in other columns.
* "ozone" and "windGust" columns were dropped because they were missing so much information, and I had no good strategy for imputing those missing values.

In [ ]:
df.drop(columns = ['readingTime', 'weatherSummary', 'precipProbability', 'apparentTemperatureHigh',
                   'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureLowTime',
                   'uvIndexTime', 'apparentTemperatureMin', 'apparentTemperatureMinTime',
                   'apparentTemperatureMax', 'apparentTemperatureMaxTime', 'visibility', 'precipIntensityMax',
                   'dewPoint', 'temperatureHigh', 'temperatureHighTime', 'temperatureLow',
                   'temperatureLowTime', 'temperatureMinTime', 'temperatureMaxTime', 'icon', 'ozone',
                   'precipIntensityMaxTime', 'windGust', 'windGustTime'],
        inplace = True)

In [ ]:
gc.collect()

In [ ]:
df.info()

---
#### Forward fill ice thickness values until next observed value
Since ice thickness is only measured periodically, I chose to make the naive assumption that it remains constant until the next measurement.

In [ ]:
df['iceThickness'].fillna(method = 'pad', inplace = True)

In [ ]:
df.head()

In [ ]:
df['iceThickness'].isna().sum()

In [ ]:
df.loc[df['iceThickness'].isna() == False]

The first 56 days' values were missed by the first fill operation; I backfilled them to eliminate NaN values in the iceThickness column.

In [ ]:
df['iceThickness'].fillna(method = 'bfill', inplace = True)

In [ ]:
df['iceThickness'].isna().sum()

In [ ]:
df['iceThickness'].value_counts()

---
#### Display DataFrame information
I wanted an idea of what other missing data I needed to deal with.

In [ ]:
df.info()

---
#### temperatureMin, temperatureMax
There were 112 missing values in these columns. I decided to fill in missing values with the mean of the values before and after a missing value.

In [ ]:
# for each year in the data, fill missing values with mean for previous known and next known values
year_list = [year for year in df['Date'].dt.year.unique()]
for year in year_list:
    temp_df = df.loc[df['Date'].dt.year == year]
    gap_fill_mean(temp_df, 'temperatureMin')
    gap_fill_mean(temp_df, 'temperatureMax')
    for idx in temp_df.index:
        df['temperatureMin'].loc[idx] = temp_df['temperatureMin'].loc[idx]
        df['temperatureMax'].loc[idx] = temp_df['temperatureMax'].loc[idx]

In [ ]:
df.info()

---
#### precipAccumulation
Since precipAccumulation is defined as "The amount of snowfall accumulation expected to occur (over the hour or day, respectively), in inches. (If no snowfall is expected, this property will not be defined.)," I decided to fill missing values in that column with 0.

In [ ]:
df['precipAccumulation'].isna().sum()

In [ ]:
df['precipAccumulation'].fillna(value = 0, downcast = 'infer', inplace = True)

In [ ]:
df['precipAccumulation'].isna().sum()

In [ ]:
df.info()

---
#### precipType
I wanted to keep this column since rain or snow could impact the river ice, so I filled NaNs with 'None'.

In [ ]:
df['precipType'].value_counts()

In [ ]:
df['precipType'].fillna(value = 'None', inplace = True)

In [ ]:
df.info()

---
#### winningTime
I filled missing values in 'winningTime' with zeroes, since there will only be an entry in this column for the day that the ice broke.

In [ ]:
df['winningTime'].fillna(value = 0, inplace = True)

In [ ]:
df.info()

---
#### atmoPressure
Barometric pressure is used as an aid in forecasting weather. Falling pressure indicates incoming inclement weather and rising pressure indicates incoming fair weather. With temperature, precipitation, and UV index information available, I chose to drop this column.

In [ ]:
df.drop(columns = 'atmoPressure', inplace = True)

---
#### windSpeed, windBearing
I decided to replace missing values in these columns with zeroes.

In [ ]:
df['windSpeed'].fillna(value = 0, inplace = True)
df['windBearing'].fillna(value = 0, inplace = True)

In [ ]:
df.info()

---
#### humidity
I decided to take the known value before a gap, and the known value after a gap, and fill between with the mean of the two values.

In [ ]:
df['humidity'].isna().sum()

In [ ]:
gap_fill_mean(df, 'humidity')

In [ ]:
df['humidity'].isna().sum()

In [ ]:
df.info()

---
#### cloudCover
After looking at the records surrounding the missing values, I decided to take the known value before a gap, and the known value after a gap, and fill between with the mean of the two values.

In [ ]:
# how many missing values?
df['cloudCover'].isna().sum()

In [ ]:
gap_fill_mean(df, 'cloudCover')

In [ ]:
df['cloudCover'].isna().sum()

In [ ]:
df.info()

---
#### precipIntensity
This is defined as "the intensity (in inches of liquid water per hour) of precipitation occurring at the given time. This value is conditional on probability (that is, assuming any precipitation occurs at all)," so I replaced missing values with zeroes.

In [ ]:
df['precipIntensity'].fillna(value = 0, inplace = True)

In [ ]:
df.info()

---
#### uvIndex
This was another case where the number of missing values was small (39), comprising 0.88% of the total values. I decided to replace the missing values with the median.

In [ ]:
df['uvIndex'].isna().sum()

In [ ]:
gap_fill_mean(df, 'uvIndex')

In [ ]:
df.info()

### Save cleaned data to file

In [ ]:
df.to_csv('../data/cleaned_data.csv', index_label = 'Date')

### Read data from file
Start here once the data has been cleaned and saved.

In [4]:
# read data
df = pd.read_csv('../data/cleaned_data.csv')

In [5]:
df.head()

,Date,Date.1,sunriseTime,sunsetTime,moonPhase,humidity,windSpeed,windBearing,cloudCover,uvIndex,temperatureMin,temperatureMax,precipType,precipIntensity,precipAccumulation,winningTime,iceThickness,flowVolume
0,0,1989-01-01,599687760.0,599702820.0,0.81,0.83,4.67,148.0,0.18,0.0,3.68,25.80,None,0.0,0.0,0,42.0,7600
1,1,1989-01-02,599774040.0,599789340.0,0.84,0.80,4.30,131.0,0.16,0.0,-8.27,24.93,None,0.0,0.0,0,42.0,7600
2,2,1989-01-03,599860380.0,599875920.0,0.88,0.69,2.26,104.0,0.22,0.0,-10.28,4.69,None,0.0,0.0,0,42.0,7600
3,3,1989-01-04,599946720.0,599962440.0,0.91,0.60,7.51,69.0,0.17,0.0,-16.58,7.67,None,0.0,0.0,0,42.0,7600
4,4,1989-01-05,600033000.0,600048960.0,0.94,0.71,7.30,78.0,0.13,0.0,2.57,14.70,None,0.0,0.0,0,42.0,7600


In [6]:
df.drop(columns = 'Date', inplace = True)

In [7]:
df.rename(columns = {'Date.1' : 'Date'}, inplace = True)

In [8]:
df['Date'] = pd.to_datetime(df['Date'], yearfirst = True, infer_datetime_format = True)

In [9]:
df.shape

(4456, 17)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4456 entries, 0 to 4455
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                4456 non-null   datetime64[ns]
 1   sunriseTime         4456 non-null   float64       
 2   sunsetTime          4456 non-null   float64       
 3   moonPhase           4456 non-null   float64       
 4   humidity            4456 non-null   float64       
 5   windSpeed           4456 non-null   float64       
 6   windBearing         4456 non-null   float64       
 7   cloudCover          4456 non-null   float64       
 8   uvIndex             4456 non-null   float64       
 9   temperatureMin      4456 non-null   float64       
 10  temperatureMax      4456 non-null   float64       
 11  precipType          4456 non-null   object        
 12  precipIntensity     4456 non-null   float64       
 13  precipAccumulation  4456 non-null   float64     

## Feature Engineering

In [11]:
# create column for ordinal day of year
df['dayOfYear'] = df['Date'].dt.dayofyear

In [14]:
df['winningTime'].value_counts()

0                      4427
2020-03-30 15:48:00       1
2020-03-30 21:27:00       1
2020-03-30 13:00:00       1
2020-03-30 22:53:00       1
2020-03-30 09:06:00       1
2020-03-30 14:25:00       1
2020-03-30 18:22:00       1
2020-03-30 14:16:00       1
2020-03-30 16:54:00       1
2020-03-30 19:39:00       1
2020-03-30 13:18:00       1
2020-03-30 20:41:00       1
2020-03-30 12:01:00       1
2020-03-30 17:29:00       1
2020-03-30 10:28:00       1
2020-03-30 16:24:00       1
2020-03-30 10:47:00       1
2020-03-30 23:01:00       1
2020-03-30 17:19:00       1
2020-03-30 12:00:00       1
2020-03-30 14:41:00       1
2020-03-30 13:01:00       1
2020-03-30 00:04:00       1
2020-03-30 15:39:00       1
2020-03-30 06:26:00       1
2020-03-30 20:14:00       1
2020-03-30 12:32:00       1
2020-03-30 00:21:00       1
2020-03-30 15:47:00       1
Name: winningTime, dtype: int64

In [17]:
# create binary 'winningDate' column
# if processed from start of notebook, winningTime value will be 0; if loaded from file, winningTime is '0'
df['winningDate'] = 0
idx = df.loc[df['winningTime'] != '0'].index
df['winningDate'].loc[idx] = 1

In [18]:
df['winningDate'].value_counts()

0    4427
1      29
Name: winningDate, dtype: int64

In [19]:
df.loc[df['winningDate'] == 1]

,Date,sunriseTime,sunsetTime,moonPhase,humidity,windSpeed,windBearing,cloudCover,uvIndex,temperatureMin,temperatureMax,precipType,precipIntensity,precipAccumulation,winningTime,iceThickness,flowVolume,dayOfYear,winningDate
108,1989-05-01,6.100323e+08,6.100934e+08,0.87,0.47,3.72,277.0,0.140,4.0,36.22,61.53,None,0.0000,0.0,2020-03-30 20:14:00,40.0,62000,121,1
252,1990-04-24,6.409650e+08,6.410233e+08,0.00,0.61,0.00,0.0,0.160,4.0,28.66,44.74,None,0.0000,0.0,2020-03-30 17:19:00,36.0,15000,114,1
400,1991-05-01,6.731044e+08,6.731654e+08,0.60,0.49,4.13,139.0,0.100,4.0,31.43,57.62,None,0.0000,0.0,2020-03-30 00:04:00,38.0,40000,121,1
556,1992-05-14,7.058471e+08,7.059137e+08,0.45,0.49,5.83,239.0,0.320,5.0,30.52,51.11,None,0.0000,0.0,2020-03-30 06:26:00,47.0,18000,135,1
684,1993-04-23,7.355732e+08,7.356311e+08,0.07,0.52,7.72,330.0,0.020,4.0,31.28,45.50,None,0.0000,0.0,2020-03-30 13:01:00,25.0,19000,113,1
839,1994-04-29,7.676264e+08,7.676866e+08,0.67,0.31,4.39,22.0,0.130,4.0,26.51,59.54,None,0.0000,0.0,2020-03-30 23:01:00,51.0,42000,119,1
1007,1996-05-05,8.313018e+08,8.313647e+08,0.60,0.44,4.42,64.0,0.380,4.0,32.47,53.42,None,0.0000,0.0,2020-03-30 12:32:00,45.0,27000,126,1
1153,1997-04-30,8.624069e+08,8.624677e+08,0.79,0.48,4.05,252.0,0.565,1.5,25.75,61.35,None,0.0000,0.0,2020-03-30 10:28:00,36.0,17000,120,1
1291,1998-04-20,8.930811e+08,8.931377e+08,0.80,0.60,4.06,287.0,0.350,3.0,25.97,52.56,rain,0.0000,0.0,2020-03-30 16:54:00,38.0,15500,110,1
1577,2000-05-01,9.571874e+08,9.572488e+08,0.93,0.46,4.97,211.0,0.880,4.0,34.92,55.95,None,0.0000,0.0,2020-03-30 10:47:00,36.0,20000,122,1


The information for 1995 and 1999 are missing some dates, including the winning date. I decided to drop the data for those years, since there is no target as a result. Survival analysis would also falsely treat those years as censored.

In [20]:
drop_1995_1999 = df.loc[(df['Date'].dt.year == 1995)|(df['Date'].dt.year == 1999)].index

In [21]:
df.drop(index = drop_1995_1999, inplace = True)

Drop all records for a year that are after the ice broke.

In [22]:
year_list = sorted(list(set(df['Date'].dt.year)))

In [23]:
print(year_list)

[1989, 1990, 1991, 1992, 1993, 1994, 1996, 1997, 1998, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [24]:
# get winning date indices
idx_w = df.loc[df['winningDate'] == 1].index
idx_w

Int64Index([ 108,  252,  400,  556,  684,  839, 1007, 1153, 1291, 1577, 1735,
            1875, 2003, 2150, 2305, 2460, 2606, 2767, 2913, 3062, 3218, 3359,
            3537, 3663, 3813, 3964, 4123, 4274, 4408],
           dtype='int64')

In [25]:
# make a list of indices to drop
drop_index = []
for i, year in enumerate(year_list):
    idx_y = df.loc[df['Date'].dt.year == year].index
    for idx in idx_y:
        if idx > idx_w[i]:
            drop_index.append(idx)
        else:
            pass

In [26]:
# drop observations that occurred after the winning date in a year
for idx in drop_index:
    df.drop(index = idx, inplace = True)

In [27]:
df.shape

(3336, 19)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3336 entries, 0 to 4408
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                3336 non-null   datetime64[ns]
 1   sunriseTime         3336 non-null   float64       
 2   sunsetTime          3336 non-null   float64       
 3   moonPhase           3336 non-null   float64       
 4   humidity            3336 non-null   float64       
 5   windSpeed           3336 non-null   float64       
 6   windBearing         3336 non-null   float64       
 7   cloudCover          3336 non-null   float64       
 8   uvIndex             3336 non-null   float64       
 9   temperatureMin      3336 non-null   float64       
 10  temperatureMax      3336 non-null   float64       
 11  precipType          3336 non-null   object        
 12  precipIntensity     3336 non-null   float64       
 13  precipAccumulation  3336 non-null   float64     

### Calculate the number of hours of daylight
Daylight hours fluctuate a lot in Alaska. Articles I had read about the NIC mention that things like how much snow covers the ice, and therefore how much direct sunlight the ice receives, affect the rate the river ice melts.

I proxied snow depth with precipitation measurements. I theorized that the number of sunlight hours in a day may be a proxy for how much sunlight the river ice receives. I also had information regarding cloud cover which may help in that regard.

I calculated the number of daylight hours as a percentage of the day. That way its value is already scaled for use in modeling later.

In [29]:
def calc_daylight(sunrise, sunset):
    daylight_hours = (sunset - sunrise) / 60 / 60 /24
    return daylight_hours

In [30]:
df['daylightHours'] = calc_daylight(df['sunriseTime'], df['sunsetTime'])

In [31]:
df.head()

,Date,sunriseTime,sunsetTime,moonPhase,humidity,windSpeed,windBearing,cloudCover,uvIndex,temperatureMin,temperatureMax,precipType,precipIntensity,precipAccumulation,winningTime,iceThickness,flowVolume,dayOfYear,winningDate,daylightHours
0,1989-01-01,599687760.0,599702820.0,0.81,0.83,4.67,148.0,0.18,0.0,3.68,25.80,None,0.0,0.0,0,42.0,7600,1,0,0.174306
1,1989-01-02,599774040.0,599789340.0,0.84,0.80,4.30,131.0,0.16,0.0,-8.27,24.93,None,0.0,0.0,0,42.0,7600,2,0,0.177083
2,1989-01-03,599860380.0,599875920.0,0.88,0.69,2.26,104.0,0.22,0.0,-10.28,4.69,None,0.0,0.0,0,42.0,7600,3,0,0.179861
3,1989-01-04,599946720.0,599962440.0,0.91,0.60,7.51,69.0,0.17,0.0,-16.58,7.67,None,0.0,0.0,0,42.0,7600,4,0,0.181944
4,1989-01-05,600033000.0,600048960.0,0.94,0.71,7.30,78.0,0.13,0.0,2.57,14.70,None,0.0,0.0,0,42.0,7600,5,0,0.184722


In [32]:
df['daylightHours'].value_counts()

0.184722    13
0.188194    13
0.179167    12
0.174306    12
0.177778    11
            ..
0.443750     1
0.273611     1
0.726389     1
0.302778     1
0.364583     1
Name: daylightHours, Length: 748, dtype: int64

In [33]:
df.drop(columns = ['sunriseTime', 'sunsetTime'], inplace = True)

In [34]:
df['past'] = (df['Date'] < '2015-01-01').astype(np.int)

In [35]:
df['future'] = 1 - df['past']

In [36]:
df.head(3).append(df.tail(3))

,Date,moonPhase,humidity,windSpeed,windBearing,cloudCover,uvIndex,temperatureMin,temperatureMax,precipType,precipIntensity,precipAccumulation,winningTime,iceThickness,flowVolume,dayOfYear,winningDate,daylightHours,past,future
0,1989-01-01,0.81,0.83,4.67,148.0,0.18,0.0,3.68,25.80,None,0.0000,0.0,0,42.0,7600,1,0,0.174306,1,0
1,1989-01-02,0.84,0.80,4.30,131.0,0.16,0.0,-8.27,24.93,None,0.0000,0.0,0,42.0,7600,2,0,0.177083,1,0
2,1989-01-03,0.88,0.69,2.26,104.0,0.22,0.0,-10.28,4.69,None,0.0000,0.0,0,42.0,7600,3,0,0.179861,1,0
4406,2019-04-12,0.27,0.60,2.50,322.0,0.84,2.0,33.09,46.88,None,0.0000,0.0,0,25.3,11400,102,0,0.616667,0,1
4407,2019-04-13,0.30,0.70,2.14,16.0,0.90,2.0,33.09,49.34,rain,0.0007,0.0,0,25.3,12100,103,0,0.622222,0,1
4408,2019-04-14,0.34,0.67,4.90,338.0,0.54,2.0,32.54,47.91,rain,0.0003,0.0,2020-03-30 00:21:00,25.3,12700,104,1,0.626389,0,1


In [37]:
df['precipType'].value_counts()

None    2657
snow     627
rain      52
Name: precipType, dtype: int64

In [38]:
# encode precipType
df = df.merge(pd.get_dummies(data = df['precipType'], prefix = 'precip', drop_first = True, sparse = True),
              how = 'left', left_index = True, right_index = True)

In [39]:
# drop precipType after encoding
df.drop(columns = 'precipType', inplace = True)
gc.collect()

0

In [40]:
df.head()

,Date,moonPhase,humidity,windSpeed,windBearing,cloudCover,uvIndex,temperatureMin,temperatureMax,precipIntensity,...,winningTime,iceThickness,flowVolume,dayOfYear,winningDate,daylightHours,past,future,precip_rain,precip_snow
0,1989-01-01,0.81,0.83,4.67,148.0,0.18,0.0,3.68,25.80,0.0,...,0,42.0,7600,1,0,0.174306,1,0,0,0
1,1989-01-02,0.84,0.80,4.30,131.0,0.16,0.0,-8.27,24.93,0.0,...,0,42.0,7600,2,0,0.177083,1,0,0,0
2,1989-01-03,0.88,0.69,2.26,104.0,0.22,0.0,-10.28,4.69,0.0,...,0,42.0,7600,3,0,0.179861,1,0,0,0
3,1989-01-04,0.91,0.60,7.51,69.0,0.17,0.0,-16.58,7.67,0.0,...,0,42.0,7600,4,0,0.181944,1,0,0,0
4,1989-01-05,0.94,0.71,7.30,78.0,0.13,0.0,2.57,14.70,0.0,...,0,42.0,7600,5,0,0.184722,1,0,0,0


Create column for daily average temperature

In [41]:
df['temperatureAvg'] = (df['temperatureMin'] + df['temperatureMax']) / 2

Create columns for number of "hot days," "cold days," and snow accumulated since Apr 1 in a given year.

I defined a "hot day" as a day where: day_average_temp > median(year_avg_temp) + std_dev(year_avg_temp)

A "cold day" is a day where: day_average_temp < median(year_avg_temp) - std_dev(year_avg_temp)

In [42]:
hot_count = []
cold_count = []
daily_accumulation = []
for year in year_list:
    hot_temp_count = 0
    cold_temp_count = 0
    daily_accum = 0
    temp_df = df.loc[df['Date'].dt.year == year]
    hot_threshold = temp_df['temperatureAvg'].median() + temp_df['temperatureAvg'].std()
    cold_threshold = temp_df['temperatureAvg'].median() - temp_df['temperatureAvg'].std()
    for idx in temp_df.index:
        current_temp = temp_df['temperatureAvg'].loc[idx]
        if temp_df['precip_snow'].loc[idx] == 1:
            daily_accum += temp_df['precipAccumulation'].loc[idx]
        else:
            pass
        if current_temp >= hot_threshold:
            hot_temp_count += 1
        elif current_temp <= cold_threshold:
            cold_temp_count += 1
        else:
            pass
        hot_count.append(hot_temp_count)
        cold_count.append(cold_temp_count)
        daily_accumulation.append(daily_accum)

In [43]:
df['numHotDays'] = hot_count
df['numColdDays'] = cold_count
df['accumulatedSnow'] = daily_accumulation

#### Save data before adding rolling average features.

In [ ]:
df.to_csv('../data/pre-moving-average_data.csv', index = False)

Create columns for moving average features.

In [44]:
ma_cols = ['humidity', 'windSpeed', 'windBearing', 'cloudCover','uvIndex', 'precipIntensity',
           'iceThickness', 'temperatureAvg', 'numHotDays', 'numColdDays', 'flowVolume']
windows = [3, 5, 7, 10]

In [45]:
# # first add new columns with dummy info
for col in ma_cols:
    for window in windows:
        label_ma = col + '_MA_' + str(window)
        df[label_ma] = 0
        
        label_slope = col + '_MA-slope_' + str(window)
        df[label_slope] = 0
        
        label_accel = col + '_MA-accel_' + str(window)
        df[label_accel] = 0
        
        label_std = col + '_MA-std_dev_' + str(window)
        df[label_std] = 0

In [46]:
df.shape

(3336, 201)

In [47]:
# Update each year with its rolling averages
# this takes ~2.5 hours to run on my laptop
for year in year_list:
    temp_df = df.loc[df['Date'].dt.year == year]
    for col in ma_cols:
        for window in windows:
            # assign labels
            label_ma = col + '_MA_' + str(window)
            label_slope = col + '_MA-slope_' + str(window)
            label_accel = col + '_MA-accel_' + str(window)
            label_std = col + '_MA-std_dev_' + str(window)
            # for each year, update row values in new columns
            for idx in temp_df.index:
                df[label_ma].loc[idx] = temp_df[col].rolling(window).mean().loc[idx]
                df[label_slope].loc[idx] = temp_df[col].rolling(window).apply(lambda x: slope(x)).loc[idx]
                df[label_accel].loc[idx] = temp_df[col].rolling(window).apply(lambda x: accel(x)).loc[idx]
                df[label_std].loc[idx] = temp_df[col].rolling(window).std().loc[idx]

In [48]:
df.sample(7)

,Date,moonPhase,humidity,windSpeed,windBearing,cloudCover,uvIndex,temperatureMin,temperatureMax,precipIntensity,...,flowVolume_MA-accel_5,flowVolume_MA-std_dev_5,flowVolume_MA_7,flowVolume_MA-slope_7,flowVolume_MA-accel_7,flowVolume_MA-std_dev_7,flowVolume_MA_10,flowVolume_MA-slope_10,flowVolume_MA-accel_10,flowVolume_MA-std_dev_10
3737,2015-02-07,0.63,0.69,1.07,48.0,0.010,0.0,-41.40,-9.47,0.0000,...,7.142857e+01,44.721360,7142.857143,-2.142857e+01,1.190476e+01,53.452248,7179.0,-2.393939e+01,7.765152e+00,78.662429
578,1993-01-05,0.42,0.87,6.50,135.0,0.370,0.0,17.50,34.57,0.0000,...,-1.605685e-12,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096,1997-03-04,0.85,0.23,9.85,87.0,0.565,0.0,12.38,30.55,0.0000,...,3.010660e-12,0.000000,5800.000000,-4.767048e-13,-7.102865e-13,0.000028,5800.0,5.387382e-13,-1.515466e-13,0.000000
3649,2014-04-11,0.40,0.50,1.94,109.0,0.000,3.0,-4.61,33.89,0.0000,...,2.207817e-12,0.000000,7200.000000,-7.627276e-13,-1.657335e-12,0.000000,7180.0,9.696970e+00,-1.515152e+01,42.163702
597,1993-01-24,0.08,0.58,1.70,28.0,0.000,0.0,-56.68,-45.53,0.0000,...,-1.605685e-12,0.000000,7400.000000,-9.534095e-13,-1.736256e-12,0.000000,7400.0,4.309905e-13,-4.849490e-13,0.000000
1939,2003-02-09,0.27,0.87,6.46,79.0,0.780,0.0,31.58,39.76,0.0000,...,7.142857e+01,109.544512,7857.142857,3.571429e+01,5.952381e+01,97.590007,7840.0,1.939394e+01,3.030303e+01,84.327404
4348,2019-02-13,0.30,0.76,5.69,271.0,0.990,0.0,5.69,23.01,0.0029,...,3.571429e+00,5.477226,7157.142857,-3.214286e+00,1.785714e+00,7.559289,7168.0,-6.303030e+00,5.871212e+00,22.010099


In [49]:
df.shape

(3336, 201)

#### Save results to file

In [50]:
df.to_csv('../data/ma_features_added.csv', index = False)

#### Read data from file

In [ ]:
df = pd.read_csv('../data/ma_features_added.csv')

In [ ]:
df.head()

### Drop columns that are highly correlated
* temperatureMin and temperatureMax information was captured in temperatureAvg
* precipAccumulation information was captured in accumulatedSnow

In [51]:
df.drop(columns = ['temperatureMin', 'temperatureMax', 'precipAccumulation'],
        inplace = True)

In [52]:
gc.collect()

21

### Create training and testing DataFrames

In [53]:
train = df.loc[df['past'] == 1]
train.drop(columns = ['past', 'future'], inplace = True)

In [54]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2762 entries, 0 to 3663
Columns: 196 entries, Date to flowVolume_MA-std_dev_10
dtypes: Sparse[int64, 0](2), datetime64[ns](1), float64(187), int64(5), object(1)
memory usage: 4.1+ MB


In [55]:
test = df.loc[df['future'] == 1]
test.drop(columns = ['past', 'future'], inplace = True)

In [56]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574 entries, 3700 to 4408
Columns: 196 entries, Date to flowVolume_MA-std_dev_10
dtypes: Sparse[int64, 0](2), datetime64[ns](1), float64(187), int64(5), object(1)
memory usage: 876.4+ KB


In [57]:
train.dropna(inplace = True)

In [58]:
test.dropna(inplace = True)

In [59]:
df.drop(columns = ['past', 'future'], inplace = True)
gc.collect()

0

#### Save training and testing DataFrames to file

In [60]:
train.to_csv('../data/model_training_data.csv', index = False)

In [61]:
test.to_csv('../data/model_testing_data.csv', index = False)